In [1]:
#!/usr/bin/python
import psycopg2
import numpy as np
import time


def CreateTables(cur):
    for command in create_tables:
        cur.execute(command)

def DropTables(cur):
    for command in drop_tables:
        cur.execute(command)

def getNumber(cur,region):
    get_number = [
    """
    select l.label_type_id, count(*)
    from sidewalk.label l, sidewalk.problem_severity ps, sidewalk.label_point p, reg_lab rl
    where l.label_id = ps.label_id and ps.label_id = p.label_id and l.label_id = rl.label_id and rl.region_id =%d
    group by l.label_type_id;
    """%(region)
    ]

    for command in get_number:
        cur.execute(command)
        rows = cur.fetchall()
        num = np.zeros(7)
        for row in rows:
            num[int(row[0])-1]=int(row[1])
    return num


def getAllPoints(cur, region):
    get_all_points = []
    for i in range(7):
        get_all_points.append(
        """
        select distinct ps.label_id, ps.severity
        from sidewalk.label l, sidewalk.problem_severity ps, sidewalk.label_point p, reg_lab rl
        where l.label_id = ps.label_id and ps.label_id = p.label_id and l.label_type_id = %d
        and l.label_id = rl.label_id and rl.region_id =%d
        order by ps.severity DESC
        """%(i+1,region)
        )

    allPoint = []
    for command in get_all_points:
        cur.execute(command)
        rows = cur.fetchall()
        allPoint.append(np.zeros(len(rows)))
        index = 0
        for row in rows:
            allPoint[-1][index] = np.array(row[0])
            index = index+1
    return allPoint

def calculateZoomLevel(LABEL_TYPE,ZOOM_LEVEL,allPoints,VisNum):
    zoomLevel = []
    for i in range(LABEL_TYPE):
        for j in range(allPoints[i].shape[0]):
            for z in range(ZOOM_LEVEL):
                if j<=VisNum[z,i]:
                    zoomLevel.append([int(allPoints[i][j]),z])
    return zoomLevel

def addZoomLevel(cur,zoomLevel):
    for point in zoomLevel:
        cur.execute("INSERT INTO sidewalk.label_presampled VALUES (%d, %d);"%(point[0],point[1]))

def seperateTables(cur,ZOOM_LEVEL):
    for z in range(ZOOM_LEVEL):
        cur.execute(
        """
        INSERT INTO sidewalk.label_presampled_z%d(
        SELECT ps.label_id
        FROM sidewalk.label_presampled ps
        WHERE ps.zoom_level = %d
        );
        """%(z,z)
        )

def clean_tables(cur):
    cur.execute(
    """
    truncate table label_presampled;
    """
    )

def query(cur):
    queries = [
    """
    SELECT l.label_id
    FROM sidewalk.label l, sidewalk.label_presampled lp
    WHERE l.label_id = lp.label_id and lp.zoom_level = 6
    and l.panorama_lat>38.87 and l.panorama_lat<38.95
    and l.panorama_lng>-77.5 and l.panorama_lng<-77;
    """
    ]

def buildRtree(cur):
    cur.execute(
    """
    DROP INDEX IF EXISTS rt;
    CREATE INDEX rt ON sidewalk.label(panorama_lat,panorama_lng);
    """
    )

def dropRtree(cur):
    cur.execute(
    """
    DROP INDEX rt ON sidewalk.label;
    """
    )


def test_Index(cur):
    begin_time = time.time()
    for i in range(100):
        query(cur)
    end_time = time.time()
    print("Query time without index= ", end_time - begin_time)

    buildRtree(cur)

    begin_time = time.time()
    for i in range(100):
        query(cur)
    end_time = time.time()
    print("Query time with index = ", end_time - begin_time)

def create_reg_lab(cur):
    print("Building reg_lab table...")
    queries = [
    """
    DROP TABLE IF EXISTS reg_lab;
    CREATE TABLE reg_lab(label_id int,region_id int);
    insert into reg_lab
    SELECT label.label_id,max(region.region_id)
    FROM sidewalk.street_edge
    INNER JOIN sidewalk.region
    ON ST_Intersects(street_edge.geom, region.geom)
    INNER JOIN sidewalk.audit_task
    ON audit_task.street_edge_id = street_edge.street_edge_id
    INNER JOIN sidewalk.label
    ON label.audit_task_id=audit_task.audit_task_id
    WHERE region.deleted = FALSE AND street_edge.deleted=FALSE
    GROUP BY label.label_id;
    """
    ]
    for query in queries:
        cur.execute(query)


def getRegion(cur):
    print("Geting distinct region ids")
    cur.execute(
    """
    SELECT rl.region_id
    FROM reg_lab rl
    GROUP BY rl.region_id
    ORDER BY rl.region_id
    """
    )
    region = []
    rows = cur.fetchall()
    for row in rows:
        region.append(int(row[0]))
    return region


In [ ]:
ZOOM_LEVEL = 8
LABEL_TYPE = 7
SampDeg = 0.6
try:
    conn = psycopg2.connect("dbname='sidewalk' user='sidewalk' host='localhost' port='5433' password='sidewalk'")
except psycopg2.Error:
    print("I am unable to connect to the database")
cur = conn.cursor()
clean_tables(cur)
create_reg_lab(cur)
regions = getRegion(cur)



Building reg_lab table...
Geting distinct region ids


In [ ]:
totalVisNum = []
#print(regions)
for region in regions:
    print("processing region:", region)
    totalNum = getNumber(cur,region)
    VisNum = np.zeros((ZOOM_LEVEL,LABEL_TYPE))
    VisNum[ZOOM_LEVEL-1] = totalNum
    for i in range(ZOOM_LEVEL-1):
        VisNum[ZOOM_LEVEL-2-i] = VisNum[ZOOM_LEVEL-1-i]*SampDeg
    VisNum.astype(int)
    totalVisNum.append(VisNum)
    if region == 205:
        np.savetxt("205_better.csv", VisNum, delimiter=",")
    allPoints = getAllPoints(cur,region)
    zoomLevel = calculateZoomLevel(LABEL_TYPE,ZOOM_LEVEL,allPoints,VisNum)
    addZoomLevel(cur,zoomLevel)


processing region: 195
processing region: 196
processing region: 197
processing region: 198
processing region: 199
processing region: 200
processing region: 201
processing region: 202
processing region: 203
processing region: 204
processing region: 205
processing region: 206
processing region: 207
processing region: 208
processing region: 209
processing region: 210
processing region: 211
processing region: 212
processing region: 213
processing region: 214
processing region: 215
processing region: 216
processing region: 217
processing region: 218
processing region: 219
processing region: 220
processing region: 221
processing region: 222
processing region: 223
processing region: 224
processing region: 225
processing region: 226
processing region: 227
processing region: 228
processing region: 229
processing region: 230
processing region: 231
processing region: 232
processing region: 233
processing region: 234
processing region: 235
processing region: 236
processing region: 237
processing 

In [5]:
np.savetxt("total_better.csv", sum(totalVisNum), delimiter=",")

In [ ]:
cur.execute("select * from sidewalk.label_presampled_z6 limit 20;")
rows = cur.fetchall()
rows

In [ ]:
cur.execute(
"""
select rl.region_id,count(*)
from reg_lab rl
group by rl.region_id;
"""
)
rows = cur.fetchall()
rows

In [9]:
a= np.genfromtxt('total_better.csv',delimiter=',').astype(int)
a

array([[ 2599,   320,   298,   192,     9,     0,     0],
       [ 4332,   534,   496,   320,    15,     0,     0],
       [ 7221,   890,   827,   533,    24,     0,     0],
       [12034,  1483,  1378,   888,    41,     0,     0],
       [20057,  2472,  2296,  1480,    68,     0,     1],
       [33428,  4121,  3827,  2467,   113,     0,     1],
       [55714,  6868,  6379,  4111,   189,     0,     2]])